In [4]:
import os
from PIL import Image

src_folder = r"C:\Users\yalda\OneDrive\Desktop\Tpose Coraline Stylle\wave dancing clay"
dst_folder = src_folder + "_resized"
os.makedirs(dst_folder, exist_ok=True)

size = (512, 512)  

for file in os.listdir(src_folder):
    if file.lower().endswith(".png"):
        img = Image.open(os.path.join(src_folder, file)).convert("RGB")
        img = img.resize(size, Image.LANCZOS)
        img.save(os.path.join(dst_folder, file))

print(f"Resized images saved to {dst_folder}")


Resized images saved to C:\Users\yalda\OneDrive\Desktop\Tpose Coraline Stylle\wave dancing clay_resized


In [10]:
import os, re, random
from pathlib import Path
from PIL import Image

# ----------------- CONFIG -----------------
A_DIR = r"C:\Users\yalda\OneDrive\Desktop\Tpose Coraline Stylle\wave dancing normal_resized"
B_DIR = r"C:\Users\yalda\OneDrive\Desktop\Tpose Coraline Stylle\wave dancing clay_resized"

OUT_ROOT = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\datasets\coraline_p2p_v3"
SPLIT = (0.90, 0.05, 0.05)   
RESIZE_TO = 512              
OUT_EXT = ".png"             


def list_images(folder):
    exts = {".png", ".jpg", ".jpeg", ".webp", ".bmp"}
    return sorted([p for p in Path(folder).glob("*") if p.suffix.lower() in exts])

A_files = list_images(A_DIR)
B_files = list_images(B_DIR)


B_map = {p.stem: p for p in B_files}
pairs = [(a, B_map[a.stem]) for a in A_files if a.stem in B_map]

print(f"Found A: {len(A_files)}, B: {len(B_files)}, matched pairs: {len(pairs)}")
assert pairs, "No matching filenames between A and B!"


random.seed(42)
random.shuffle(pairs)

n = len(pairs)
n_train = int(SPLIT[0]*n)
n_val   = int(SPLIT[1]*n)
train_pairs = pairs[:n_train]
val_pairs   = pairs[n_train:n_train+n_val]
test_pairs  = pairs[n_train+n_val:]

splits = {
    "train": train_pairs,
    "val":   val_pairs,
    "test":  test_pairs
}


for split in splits:
    Path(OUT_ROOT, split).mkdir(parents=True, exist_ok=True)

def load_and_prepare(path, resize_to):
    img = Image.open(path).convert("RGB")
    if resize_to is not None:
        img = img.resize((resize_to, resize_to), Image.LANCZOS)
    else:
        w, h = img.size
        m = min(w, h)
        left = (w - m)//2
        top  = (h - m)//2
        img = img.crop((left, top, left+m, top+m))
    return img

def save_AB(a_path, b_path, out_path, resize_to):
    A = load_and_prepare(a_path, resize_to)
    B = load_and_prepare(b_path, resize_to)
    AB = Image.new("RGB", (A.width + B.width, A.height))
    AB.paste(A, (0,0))
    AB.paste(B, (A.width,0))
    AB.save(out_path)

count = {"train":0, "val":0, "test":0}
for split, plist in splits.items():
    out_dir = Path(OUT_ROOT, split)
    for a,b in plist:
        out_name = a.stem + OUT_EXT 
        out_path = out_dir / out_name
        save_AB(a, b, out_path, RESIZE_TO)
        count[split] += 1

print("Done.")
print(count)
print(f"AB dataset ready at: {OUT_ROOT}")


Found A: 479, B: 479, matched pairs: 479
Done.
{'train': 431, 'val': 23, 'test': 25}
AB dataset ready at: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\datasets\coraline_p2p_v3


In [9]:
import os

folder_path = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\datasets\coraline_p2p_v3\train"

for filename in os.listdir(folder_path):
    old_path = os.path.join(folder_path, filename)
    
    if os.path.isfile(old_path):
        new_filename = "angry_" + filename
        new_path = os.path.join(folder_path, new_filename)
        os.rename(old_path, new_path)

print("All files renamed with 'angry_' prefix.")


All files renamed with 'angry_' prefix.


In [1]:
import os, shlex, subprocess

repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = r"""
python train.py
--dataroot ./datasets/coraline_p2p_v3
--name coraline_p2p_v3_512
--model pix2pix
--dataset_mode aligned
--direction AtoB
--input_nc 3
--output_nc 3
--netG unet_256
--lambda_L1 100
--gan_mode lsgan
--preprocess resize_and_crop
--load_size 544
--crop_size 512
--batch_size 1
--gpu_ids 0
--n_epochs 20
--n_epochs_decay 30
--save_epoch_freq 5
--print_freq 50
--display_freq 200
"""
proc = subprocess.run(shlex.split(" ".join(cmd.split())), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\en

In [2]:
import os, subprocess

os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TORCH_COMPILE_DISABLE"]  = "1"
os.environ["TORCHDYNAMO_DISABLE"]    = "1"


repo_dir    = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
dataroot    = r".\datasets\coraline_p2p_v3"
name        = "coraline_p2p_v3_512"
epoch       = "l"
gpu_ids     = "0"
num_test    = "100000"
results_dir = r".\results"


preprocess  = "none"
load_size   = "512"
crop_size   = "512"

cmd = [
    "python", "test.py",
    "--dataroot", dataroot,
    "--name", name,
    "--model", "pix2pix",
    "--dataset_mode", "aligned",
    "--direction", "AtoB",
    "--epoch", epoch,
    "--gpu_ids", gpu_ids,
    "--num_test", num_test,
    "--results_dir", results_dir,
    "--netG", "unet_256",
    "--input_nc", "3",
    "--output_nc", "3",
    "--preprocess", preprocess,
    "--load_size", load_size,
    "--crop_size", crop_size,
    "--no_flip",
    "--serial_batches",
    "--eval",
]


os.chdir(repo_dir)
print("CWD:", os.getcwd())
print("Running:\n", " ".join(cmd))

proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


run_tag = f"test_{epoch}"
images_dir = os.path.abspath(os.path.join(results_dir, name, run_tag, "images"))
print("Results folder:", images_dir)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
Running:
 python test.py --dataroot .\datasets\coraline_p2p_v3 --name coraline_p2p_v3_512 --model pix2pix --dataset_mode aligned --direction AtoB --epoch latest --gpu_ids 0 --num_test 100000 --results_dir .\results --netG unet_256 --input_nc 3 --output_nc 3 --preprocess none --load_size 512 --crop_size 512 --no_flip --serial_batches --eval
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 512                           	[default: 256]
                 dataroot: .\datasets\coraline_p2p_v3    	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: l